In [4]:
%pip install --quiet -U crewai crewai_tools langchain_community crewai[google-genai]

Note: you may need to restart the kernel to use updated packages.


In [5]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
from crewai import Agent, Task, Crew, LLM

In [7]:
import os

from utils import get_gemini_api_key

gemini_api_key = get_gemini_api_key()

llm = LLM(model="gemini/gemini-2.0-flash")


## Agents

### planner

In [12]:
planner = Agent(
    role = "Content Planner",
    goal = "Plan engaging and factually content on {topic}",
    backstory = "You're working on planning a blog article "
                "about the topic: {topic}."
                "You collect information that helps the "
                "audience learn something "
                "and make informed decisions. "
                "Your worke is the basis for "
                "the content writer to write an article on this topic.",
    allow_delegation = False,
    verbose = True,
    llm = llm
)

### writer

In [13]:
writer = Agent(
    role = "Content Writer",
    goal = "Write insightful and factually accurate opinion piece on the {topic}",
    backstory = "You're working on a writing "
                "a new opinion piece on the topic: {topic}. "
                "You base your writing on the work of "
                "the Content Planner, who provides an outline "
                "and relevant context about the topic. "
                "You follow the main objectives and "
                "directions of the outline, "
                "as provide by the Content Planner. "
                "You also provide objective and impartial insights "
                "and back them up with information "
                "provide by the Content Planner. "
                "You acknowledge in your opinion piece "
                "when your statements are opinions "
                "as opposed to objective statements. ",
    allow_delegation = False,
    verbose = True,
    llm = llm
)

## editor

In [16]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True,
    llm = llm
)

## Task

### plan

In [19]:
plan = Task(
    description = (
        """
        1. Prioritize the latest trends, key players, 
           "and noteworthy news on {topic}.
        2. Identify the target audience, considering
           their interests and pain points.
        3. Develop a detailed outline including
           an introuduction, key points and call to action.
        4. Include SEO keywords and relevant data sources.
        """
    ),
    expected_output = """
        A comprehensive content plan document
        with and outline, audience analysis,
        SEO keywords, and resources
    """,
    agent = planner
)

### write

In [20]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### edit

In [21]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

### Crew

In [23]:
crew = Crew(
    agents = [planner, writer, editor],
    tasks = [plan, write, edit],
    verbose = True
)

In [27]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 7054d65c-462c-4a46-8cf2-e970ed3c665d                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Planner                                                                                         │
│                                                                                                                 │
│  Task:                                                                                                          │
│          1. Prioritize the latest trends, key players,                                                          │
│             "and noteworthy news on Artificial Intelligence.                                                    │
│          2. Identify the target audience, considering                                                           │
│             their interests and pain points.                                                                    │
│          3. Develop a detailed outline including                                                                │
│             an introuduction, key points and call to action.                                                    │
│          4. Include SEO keywords and relevant data sources.                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Planner                                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ## Content Plan: Artificial Intelligence Blog Article                                                          │
│                                                                                                                 │
│  **1. Target Audience:**                                                                                        │
│                                                                                                                 │
│  *   **Primary Audience:**                                                                                      │
│      *   Professionals in tech and related fields (e.g., software engineers, data scientists, project           │
│  managers) seeking to enhance their understanding of AI and its practical applications.                         │
│      *   Business leaders and decision-makers exploring AI solutions to improve efficiency, innovation, and     │
│  competitive advantage.                                                                                         │
│  *   **Secondary Audience:**                                                                                    │
│      *   Students and researchers interested in AI's theoretical and applied aspects.                           │
│      *   Tech enthusiasts and general readers curious about the impact of AI on society and everyday life.      │
│  *   **Interests:**                                                                                             │
│      *   Latest AI trends and breakthroughs.                                                                    │
│      *   Real-world applications and use cases.                                                                 │
│      *   Impact of AI on various industries.                                                                    │
│      *   Ethical considerations and challenges.                                                                 │
│      *   Career opportunities in AI.                                                                            │
│  *   **Pain Points:**                                                                                           │
│      *   Overwhelming amount of information and hype surrounding AI.                                            │
│      *   Difficulty understanding complex AI concepts.                                                          │
│      *   Lack of clear guidance on implementing AI solutions.                                                   │
│      *   Concerns about job displacement and ethical implications.                                              │
│                                                                                                                 │
│  **2. SEO Keywords:**                                                                                           │
│                                                                                                                 │
│  *   **Primary Keywords:**                                                                                      │
│      *   Artificial Intelligence (AI)                                                                           │
│      *   Machine Learning (ML)                                                                                  │
│      *   Deep Learning                                 

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 081f7572-dcbe-46d5-b51a-29ac070f0c6e                                                                     │
│  Agent: Content Planner                                                                                         │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│  Task: Proofread the given blog post for grammatical errors and alignment with the brand's voice.               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```markdown                                                                                                    │
│  # Unlocking the Potential of Artificial Intelligence: Trends, Players, and the Future                          │
│                                                                                                                 │
│  Artificial Intelligence (AI) is rapidly transforming industries and reshaping our lives, with projections      │
│  estimating a contribution of $15.7 trillion to the global economy by 2030. From self-driving cars to           │
│  personalized medicine, AI is permeating every aspect of our world. But what exactly is AI, and where is it     │
│  headed?                                                                                                        │
│                                                                                                                 │
│  At its core, Artificial Intelligence refers to the simulation of human intelligence in machines programmed to  │
│  perform tasks that typically require human intelligence. This encompasses a broad range of technologies,       │
│  including Machine Learning (ML), where systems learn from data without explicit programming; Deep Learning, a  │
│  subset of ML that uses artificial neural networks to analyze data; and Natural Language Processing (NLP),      │
│  which enables computers to understand and process human language. As AI continues to advance, understanding    │
│  its trends, key players, and ethical considerations becomes increasingly crucial for professionals,            │
│  businesses, and anyone interested in navigating this technological revolution.                                 │
│                                                                                                                 │
│  This article explores the latest trends, key players, and ethical considerations shaping the future of         │
│  Artificial Intelligence. It provides insights for professionals, businesses, and anyone interested in          │
│  understanding this transformative technology, offering a clear and concise overview of the AI landscape.       │
│                                                                                                                 │
│  ## The Cutting Edge: Latest Trends in AI                                                                       │
│                                                                                                                 │
│  The world of AI is dynamic, with new breakthroughs and applications emerging constantly. Staying abreast of    │
│  these trends is essential for anyone looking to leverage the power of AI.                                      │
│                                                                                                                 │
│  ### Generative AI: The Creative Revolution                                                                     │
│                                                                                                                 │
│  Generative AI is a category of Artificial Intelligence focused on creating new content, from text and images   │
│  to code and music. These models, powered by sophisticated algorithms, learn from vast datasets and then        │
│  generate original outputs that resemble the data they 

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 9b4bc5ec-fbf1-4bb0-98bd-3cdc08681df1                                                                     │
│  Agent: Editor                                                                                                  │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 7054d65c-462c-4a46-8cf2-e970ed3c665d                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ```markdown                                                                                      │
│  # Unlocking the Potential of Artificial Intelligence: Trends, Players, and the Future                          │
│                                                                                                                 │
│  Artificial Intelligence (AI) is rapidly transforming industries and reshaping our lives, with projections      │
│  estimating a contribution of $15.7 trillion to the global economy by 2030. From self-driving cars to           │
│  personalized medicine, AI is permeating every aspect of our world. But what exactly is AI, and where is it     │
│  headed?                                                                                                        │
│                                                                                                                 │
│  At its core, Artificial Intelligence refers to the simulation of human intelligence in machines programmed to  │
│  perform tasks that typically require human intelligence. This encompasses a broad range of technologies,       │
│  including Machine Learning (ML), where systems learn from data without explicit programming; Deep Learning, a  │
│  subset of ML that uses artificial neural networks to analyze data; and Natural Language Processing (NLP),      │
│  which enables computers to understand and process human language. As AI continues to advance, understanding    │
│  its trends, key players, and ethical considerations becomes increasingly crucial for professionals,            │
│  businesses, and anyone interested in navigating this technological revolution.                                 │
│                                                                                                                 │
│  This article explores the latest trends, key players, and ethical considerations shaping the future of         │
│  Artificial Intelligence. It provides insights for professionals, businesses, and anyone interested in          │
│  understanding this transformative technology, offering a clear and concise overview of the AI landscape.       │
│                                                                                                                 │
│  ## The Cutting Edge: Latest Trends in AI                                                                       │
│                                                                                                                 │
│  The world of AI is dynamic, with new breakthroughs and applications emerging constantly. Staying abreast of    │
│  these trends is essential for anyone looking to leverage the power of AI.                                      │
│                                                                                                                 │
│  ### Generative AI: The Creative Revolution                                                                     │
│                                                                                                                 │
│  Generative AI is a category of Artificial Intelligence focused on creating new content, from text and images   │
│  to code and music. These models, powered by sophistic

In [28]:
from IPython.display import Markdown

In [36]:
Markdown(result.raw)



```markdown
# Unlocking the Potential of Artificial Intelligence: Trends, Players, and the Future

Artificial Intelligence (AI) is rapidly transforming industries and reshaping our lives, with projections estimating a contribution of $15.7 trillion to the global economy by 2030. From self-driving cars to personalized medicine, AI is permeating every aspect of our world. But what exactly is AI, and where is it headed?

At its core, Artificial Intelligence refers to the simulation of human intelligence in machines programmed to perform tasks that typically require human intelligence. This encompasses a broad range of technologies, including Machine Learning (ML), where systems learn from data without explicit programming; Deep Learning, a subset of ML that uses artificial neural networks to analyze data; and Natural Language Processing (NLP), which enables computers to understand and process human language. As AI continues to advance, understanding its trends, key players, and ethical considerations becomes increasingly crucial for professionals, businesses, and anyone interested in navigating this technological revolution.

This article explores the latest trends, key players, and ethical considerations shaping the future of Artificial Intelligence. It provides insights for professionals, businesses, and anyone interested in understanding this transformative technology, offering a clear and concise overview of the AI landscape.

## The Cutting Edge: Latest Trends in AI

The world of AI is dynamic, with new breakthroughs and applications emerging constantly. Staying abreast of these trends is essential for anyone looking to leverage the power of AI.

### Generative AI: The Creative Revolution

Generative AI is a category of Artificial Intelligence focused on creating new content, from text and images to code and music. These models, powered by sophisticated algorithms, learn from vast datasets and then generate original outputs that resemble the data they were trained on. This technology is impacting content creation, code generation, and design processes across various industries.

Key players like OpenAI (with GPT-4 and DALL-E 2), Google (with Bard), and Stability AI (with Stable Diffusion) are at the forefront of Generative AI development. Use cases range from generating marketing copy and realistic images to creating functional code snippets and designing new products. The potential applications are extensive, making Generative AI a significant and rapidly evolving area of AI.

### AI Ethics and Governance: Navigating the Moral Maze

As AI systems become more powerful and pervasive, ethical considerations are increasingly important. Bias in algorithms, privacy concerns, and the potential for misuse are some of the challenges that need to be addressed. Responsible AI development and deployment are crucial to ensure that AI benefits society.

Organizations and governments are developing AI regulations and guidelines, such as the EU AI Act, to promote ethical practices and mitigate risks. AI ethics frameworks and tools are also emerging to help developers identify and address potential biases and ethical concerns in their AI systems. Focusing on AI Ethics and Governance is essential for mitigating risks associated with AI technologies.

### Edge AI: Bringing Intelligence Closer to the Source

Edge AI involves deploying AI models on edge devices, such as smartphones, IoT devices, and autonomous vehicles, rather than relying on cloud-based processing. This approach offers benefits, including lower latency, enhanced privacy, and reduced bandwidth consumption. By processing data locally, Edge AI enables real-time decision-making and reduces the need to transmit sensitive information to the cloud.

Applications of Edge AI are diverse, ranging from autonomous vehicles that react to changing road conditions to smart cameras that detect anomalies in real-time. The deployment of Edge AI presents both technical and logistical challenges, requiring specialized hardware and software to optimize performance and efficiency on resource-constrained devices.

### AI-Powered Automation: Streamlining Operations and Boosting Efficiency

AI is transforming industries by automating complex tasks and processes. AI-powered automation is being implemented across various sectors, including manufacturing, logistics, customer service, and finance, leading to increased efficiency, reduced costs, and improved accuracy.

In manufacturing, AI-powered robots can perform repetitive tasks with greater precision and speed than humans. In customer service, AI-powered chatbots can handle routine inquiries and provide instant support. While concerns about job displacement exist, many anticipate that AI-powered automation will create new opportunities and augment human capabilities.

### Explainable AI (XAI): Unveiling the Black Box

One challenge with complex AI models is their lack of transparency. Explainable AI (XAI) aims to address this by making AI decision-making processes more understandable. XAI techniques and tools, such as SHAP and LIME, help identify the factors that influence AI predictions and provide insights into how AI systems arrive at their conclusions.

XAI is particularly important in industries where decisions have significant consequences, such as healthcare and finance. By providing transparency and accountability, XAI can help build trust in AI systems and ensure their responsible use.

### Other Notable Trends

*   **AI and Cybersecurity:** AI is being used to detect and prevent cyber threats, analyze malware, and automate security operations.
*   **AI in Healthcare:** AI is contributing to advancements in diagnostics, personalized medicine, drug discovery, and robotic surgery.

## The Key Players: Shaping the AI Landscape

The AI landscape includes a diverse range of organizations, from tech giants to specialized AI companies and research institutions. These key players are driving innovation and shaping the future of AI.

### Tech Giants

*   **Google (DeepMind):** Google's DeepMind is involved in AI research, known for its work in areas such as reinforcement learning and natural language understanding.
*   **Microsoft:** Microsoft is investing in AI, integrating AI capabilities into its cloud platform, Azure, and its suite of productivity tools.
*   **Amazon:** Amazon uses AI to power its e-commerce platform, cloud services, and voice assistant, Alexa.
*   **Meta:** Meta is developing AI for social media, virtual reality, and augmented reality applications.
*   **Apple:** Apple is integrating AI into its devices and services, focusing on privacy and user experience.

### AI-Focused Companies

*   **OpenAI:** OpenAI is an AI research company known for its work in Generative AI, including GPT-4 and DALL-E 2.
*   **Anthropic:** Anthropic is developing AI models with a focus on safety and ethics, aiming to create AI systems that align with human values.
*   **NVIDIA:** NVIDIA provides hardware and software for AI, powering many AI systems.
*   **DataRobot:** DataRobot offers an AI platform that automates the process of building and deploying machine learning models.
*   **C3.ai:** C3.ai provides AI-powered enterprise software solutions for industries such as manufacturing, energy, and healthcare.

### Research Institutions

*   **MIT:** MIT conducts research in AI, focusing on areas such as robotics, computer vision, and natural language processing.
*   **Stanford:** Stanford has an AI program, focusing on areas such as machine learning, robotics, and AI ethics.
*   **Carnegie Mellon University:** Carnegie Mellon University has a history of AI research, with strengths in areas such as robotics, machine learning, and human-computer interaction.

## Noteworthy News & Developments

The AI field is constantly evolving, with new developments occurring rapidly. Recent highlights include:

*   Investments in AI research and development by tech companies.
*   The launch of AI-powered tools and platforms across industries.
*   Discussions surrounding AI ethics, bias, and privacy.
*   Government regulations and policies related to AI development and deployment.

## The Future of AI

The future of AI presents potential for transforming society and the economy. Emerging trends and technologies, such as quantum AI and neuromorphic computing, could influence the development of AI.

Addressing the challenges and opportunities that lie ahead, including ethical considerations, workforce reskilling, and the potential for misuse, is important. By focusing on responsible AI development and deployment, we can work to ensure that AI benefits humanity.

## Take Action and Stay Informed

Artificial Intelligence is a dynamic field with the potential to impact our world. To stay informed:

*   Explore AI resources and learning opportunities (Coursera, edX, Udacity, fast.ai).
*   Share your thoughts and experiences with AI in the comments section below.
*   Follow our social media channels for updates on AI.

By embracing lifelong learning and staying informed about developments in AI, you can position yourself for success in this evolving era.
```